In [22]:
!pip install wandb
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.layers import Dense,MaxPool2D,Flatten,Conv2D
import os
from tensorflow.keras import layers
from wandb.keras import WandbCallback
import wandb
from sklearn.metrics import classification_report, confusion_matrix

In [23]:
dataset_train='/content/drive/MyDrive/Flowers/Train'
dataset_test='/content/drive/MyDrive/Flowers/Test'
width=height=224
batch_size=32

In [24]:
data_generator=ImageDataGenerator(
rescale=1./255,
horizontal_flip=True,
rotation_range=20,
zoom_range=0.1,
validation_split=0.2,
)


In [25]:
train_data=data_generator.flow_from_directory(
    dataset_train,
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='training'
)
val_data=data_generator.flow_from_directory(
    dataset_train,
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='validation'
)
test_data=data_generator.flow_from_directory(
    dataset_test,
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
  
)

Found 884 images belonging to 17 classes.
Found 204 images belonging to 17 classes.
Found 272 images belonging to 17 classes.


In [26]:
base_model = tf.keras.applications.ResNet50V2(
    input_shape=(width, height, 3),
    include_top = False,
    weights = 'imagenet',
    pooling = 'max'
    )

In [27]:
base_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

In [28]:
for layer in base_model.layers[:-8]:
  layer.trainable=False

In [29]:
model = tf.keras.Sequential([
        base_model,
        Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(17, activation='sigmoid'),              
])

In [30]:
wandb.init(project="17flowers")

In [31]:
config = wandb.config
config.learning_rate = 0.001
model.compile(
    loss=tf.keras.losses.categorical_crossentropy,optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
    metrics=['accuracy']
)

In [32]:
model.fit(train_data,
          steps_per_epoch = train_data.samples / batch_size,
          validation_data = val_data,
          validation_steps = val_data.samples / batch_size,
          epochs = 10
)



Epoch 1/10
27/27 [==============================] - 26s 803ms/step - loss: 1.4763 - accuracy: 0.6052 - val_loss: 1.7119 - val_accuracy: 0.6471
Epoch 2/10
27/27 [==============================] - 21s 743ms/step - loss: 0.6271 - accuracy: 0.9163 - val_loss: 0.8988 - val_accuracy: 0.7304
Epoch 3/10
27/27 [==============================] - 20s 733ms/step - loss: 0.3510 - accuracy: 0.9695 - val_loss: 0.4241 - val_accuracy: 0.8725
Epoch 4/10
27/27 [==============================] - 21s 743ms/step - loss: 0.2090 - accuracy: 0.9887 - val_loss: 0.8842 - val_accuracy: 0.7843
Epoch 5/10
27/27 [==============================] - 21s 743ms/step - loss: 0.1655 - accuracy: 0.9910 - val_loss: 0.3559 - val_accuracy: 0.9118
Epoch 6/10
27/27 [==============================] - 21s 744ms/step - loss: 0.1152 - accuracy: 0.9966 - val_loss: 0.2355 - val_accuracy: 0.9363
Epoch 7/10
27/27 [==============================] - 21s 752ms/step - loss: 0.0853 - accuracy: 0.9955 - val_loss: 0.2480 - val_accuracy: 0.9559

In [33]:
model.evaluate(test_data)

9/9 [==============================] - 5s 563ms/step - loss: 0.5059 - accuracy: 0.8493


[0.5058785676956177, 0.8492646813392639]